<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-pose-estimation-for-swing-improvement/blob/main/MediaPipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## MediaPipe

The provided code implements the class 'MediaPipe_class.py'. There are two options for input formats. It is possible to process either just one file in *.mp4* format or all files of the same format in one selected folder. The output for the first case is a folder named 'statistics,' which includes the processed video with all points, lines, and angles for future analysis, as well as a *.csv* file with statistics for each frame, such as angles and points coordinates.

In [ ]:
!git clone https://github.com/Strojove-uceni/23206-final-pose-estimation-for-swing-improvement
!pip install mediapipe
import cv2
import mediapipe as mp
import math
import csv
import os
import datetime
import sys
sys.path.append('/content/23206-final-pose-estimation-for-swing-improvement')
from MediaPipe_class import MediaPipe_PoseEstimation

fatal: destination path '23206-final-pose-estimation-for-swing-improvement' already exists and is not an empty directory.


In [ ]:
!unzip /content/23206-final-pose-estimation-for-swing-improvement/cropped_videos.zip
!rm /content/23206-final-pose-estimation-for-swing-improvement/cropped_videos.zip

Archive:  /content/23206-final-pose-estimation-for-swing-improvement/cropped_videos.zip
  inflating: cropped_videos/output_video__qtYF0FJBWo.mp4  
  inflating: cropped_videos/output_video__rFatlaZhEQ.mp4  
  inflating: cropped_videos/output_video_04d08bM6-6U.mp4  
  inflating: cropped_videos/output_video_0PuF8duUo1A.mp4  
  inflating: cropped_videos/output_video_0rliVha14ZQ.mp4  
  inflating: cropped_videos/output_video_10lx1mFw1eY.mp4  
  inflating: cropped_videos/output_video_1a2QrnvBUuI.mp4  
  inflating: cropped_videos/output_video_1OOqvNPDsDA.mp4  
  inflating: cropped_videos/output_video_276Ec8-zeW8.mp4  
  inflating: cropped_videos/output_video_2cD6CPZhiEY.mp4  
  inflating: cropped_videos/output_video_2Tj5jJ1Kh_Q.mp4  
  inflating: cropped_videos/output_video_2Us2Zx_S6hI.mp4  
  inflating: cropped_videos/output_video_37YZm9oCFnU.mp4  
  inflating: cropped_videos/output_video_3dY8wZdR9R8.mp4  
  inflating: cropped_videos/output_video_3nG1ugrwzH8.mp4  
  inflating: cropped_videos

In [ ]:
# Set input type
input_type = 'file'

if input_type == 'file':
    # SET AN INPUT FILE in the folder cropped_videos
    input_file = 'cropped_videos/output_video_-M5SITXMA2Y.mp4'
    csv_file_name = f'{os.path.splitext(input_file)[0]}.csv'
    output_video_name = f'{os.path.splitext(input_file)[0]}_output.mp4'

    # Create 'statistics' folder if it doesn't exist
    statistics_folder = 'statistics'
    os.makedirs(statistics_folder, exist_ok=True)

    # Set the paths for csv file and output video in the 'statistics' folder
    csv_file_path = os.path.join(statistics_folder, os.path.basename(csv_file_name))
    output_video_path = os.path.join(statistics_folder, os.path.basename(output_video_name))

    video_processor = MediaPipe_PoseEstimation(input_file, csv_file_path, output_video_path)
    video_processor.process_video()

elif input_type == 'folder':
    input_folder = 'cropped_videos'
    output_folder_videos = 'result_videos'
    output_folder_csv = 'result_statistics'

    # Create output folders if they don't exist
    os.makedirs(output_folder_videos, exist_ok=True)
    os.makedirs(output_folder_csv, exist_ok=True)

    # Iterate through files in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".mp4"):
            # Set output file names
            csv_file_name = f'{os.path.splitext(filename)[0]}.csv'
            output_video_name = f'{os.path.splitext(filename)[0]}_output.mp4'
            csv_file_path = os.path.join(output_folder_csv, csv_file_name)
            output_video_path = os.path.join(output_folder_videos, output_video_name)
            file_path = os.path.join(input_folder, filename)

            video_processor = MediaPipe_PoseEstimation(file_path, csv_file_path, output_video_path)
            video_processor.process_video()
else:
    print("You have just two options: 'file' or 'folder'")

Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
An error occurred: 'NoneType' object has no attribute 'landmark'
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
An error occurred: 'NoneType' object has no attribute 'landmark'
An error occurred: 'NoneType' object has no attribute 'landmark'
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
An error occurred: 'NoneType' object has no attribute 'landmark'
An error occurred: 'NoneType' object has no attribute 'landmark'
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Null.Frames
Nul